In [1]:
# pip install pyvista vtk
import pyvista as pv
import numpy as np
from pathlib import Path

In [2]:
# --- Config: point these to your STL files ---
STL_FILES = [
    "BE_windows.stl",
    "cavity_vacuum.stl",
    "cavity_walls.stl",
]

In [3]:

# ---- Helpers ----
def read_mesh(path):
    m = pv.read(path)
    # Ensure it's surface polydata (STL typically is); clean for robust edges
    return m.clean()

def extract_edges(mesh, feature_angle=30.0):
    """
    Extracts boundary + sharp feature edges of a triangulated surface.
    feature_angle controls which dihedral edges are considered "sharp".
    """
    return mesh.extract_feature_edges(
        boundary_edges=True,
        non_manifold_edges=True,
        feature_edges=True,
        manifold_edges=False,
        feature_angle=feature_angle,
    )

def distance(p0, p1):
    return float(np.linalg.norm(np.asarray(p1) - np.asarray(p0)))

# ---- Main GUI ----
class STLInspector:
    def __init__(self, stl_paths):
        self.plotter = pv.Plotter(window_size=[1280, 800])
        self.meshes = []
        self.actors = []
        self.edge_meshes = []
        self.edge_combo = None

        # measurement state
        self.picked = []
        self.measure_actor = None
        self.measure_text_actor = None

        # Load meshes
        for path in stl_paths:
            m = read_mesh(path)
            self.meshes.append(m)

        # Add solids and keep actor refs
        cmap = [None, None, None]  # let PyVista pick defaults; customize later if you like
        for i, m in enumerate(self.meshes):
            actor = self.plotter.add_mesh(
                m,
                name=f"solid_{i}",
                show_edges=False,
                opacity=1.0,
                pickable=True,
            )
            self.actors.append(actor)

        # Extract edges for snap-to-edge picking
        self.edge_meshes = [extract_edges(m, feature_angle=30.0) for m in self.meshes]
        # Combine all edges into one polydata for simple snapping
        self.edge_combo = pv.PolyData()
        if len(self.edge_meshes) == 1:
            self.edge_combo = self.edge_meshes[0]
        else:
            self.edge_combo = self.edge_meshes[0].merge(self.edge_meshes[1:])
        # Optional: draw edges faintly (helps the user)
        self.plotter.add_mesh(
            self.edge_combo,
            name="edges_all",
            line_width=1,
            opacity=0.25,
            render_lines_as_tubes=False,
            pickable=False,
        )

        # Add checkboxes to toggle each solid
        self._add_visibility_checkboxes()

        # Enable point picking with snapping to nearest edge vertex
        self.plotter.enable_point_picking(
            callback=self._on_pick,
            show_message=True,
            use_mesh=False,   # we will manually snap to edge vertices
            show_point=True,  # show a small sphere at picked point
        )

        # Nice camera and axes
        self.plotter.add_axes()
        self.plotter.show_bounds(grid='front', location='outer', all_edges=True)

    # ---- UI widgets ----
    def _add_visibility_checkboxes(self):
        """
        Adds one checkbox per solid. Each toggles the corresponding actor's visibility.
        We place them vertically down the left side.
        """
        start_x, start_y = 10.0, 10.0
        size = 30
        pad = 40

        def make_toggle(idx):
            def _toggle(checked):
                # checked=True means "box ticked" -> show; False -> hide
                self.actors[idx].SetVisibility(bool(checked))
                # (Optional) also hide that solid's edges for clarity:
                # We keep a single combined edge mesh for snapping, but you could
                # rebuild it based on visible meshes if needed (next iterations).
            return _toggle

        for i, _ in enumerate(self.actors):
            self.plotter.add_checkbox_button_widget(
                make_toggle(i),
                value=True,  # start visible
                position=(start_x, start_y + i * pad),
                size=size,
                border_size=1,
                color_on="white",
                color_off="grey",
            )
            self.plotter.add_text(
                f"Solid {i+1}",
                position=(start_x + size + 10, start_y + i * pad + 6),
                font_size=10,
                name=f"label_{i}",
            )

    # ---- Picking & measuring ----
    def _snap_to_edge_vertex(self, world_point):
        """
        Snap an arbitrary 3D world pick to the nearest vertex on the combined edge mesh.
        This is robust and fast. Later we can upgrade to true along-edge projection.
        """
        if self.edge_combo is None or self.edge_combo.n_points == 0:
            return world_point
        idx = self.edge_combo.find_closest_point(world_point)
        return self.edge_combo.points[idx]

    def _clear_measure(self):
        if self.measure_actor is not None:
            try:
                self.plotter.remove_actor(self.measure_actor)
            except Exception:
                pass
            self.measure_actor = None

        if self.measure_text_actor is not None:
            try:
                self.plotter.remove_actor(self.measure_text_actor)
            except Exception:
                pass
            self.measure_text_actor = None

    def _draw_measure(self, p0, p1):
        self._clear_measure()
        line = pv.Line(p0, p1)
        self.measure_actor = self.plotter.add_mesh(line, line_width=3.0)
        d = distance(p0, p1)
        mid = (np.asarray(p0) + np.asarray(p1)) / 2.0
        self.measure_text_actor = self.plotter.add_point_labels(
            [mid],
            [f"{d:.3f} units"],
            point_size=0,
            font_size=14,
            shape=None,
            always_visible=True,
        )
        self.plotter.render()

    def _on_pick(self, pick):
        """
        Fired after each click. We take the 3D pick location, snap to the nearest edge vertex,
        collect two points, and draw a measurement.
        """
        if pick is None:
            return

        snapped = self._snap_to_edge_vertex(pick)
        self.picked.append(snapped)

        # Keep only the last two picks
        if len(self.picked) > 2:
            self.picked = self.picked[-2:]

        if len(self.picked) == 2:
            p0, p1 = self.picked
            self._draw_measure(p0, p1)

    def show(self):
        self.plotter.show(title="Wakis STL Inspector (v1)")

In [4]:
# Resolve only the STL files that exist, so the script is friendly to quick testing
existing = [p for p in map(Path, STL_FILES) if p.exists()]
if not existing:
    # If you don't have STLs at hand, create quick dummy solids to test the UI.
    # Comment this block out once you use real files.
    cube = pv.Cube(center=(0, 0, 0), x_length=1.0, y_length=1.0, z_length=1.0)
    sphere = pv.Sphere(center=(2, 0, 0), radius=0.7, theta_resolution=40, phi_resolution=40)
    cone = pv.Cone(center=(-2, 0, 0), height=1.5, radius=0.6, direction=(0, 1, 0))
    cube.save("solid_a.stl"); sphere.save("solid_b.stl"); cone.save("solid_c.stl")
    existing = [Path("solid_a.stl"), Path("solid_b.stl"), Path("solid_c.stl")]

app = STLInspector(existing)
app.show()

/home/edelafue/miniconda3/envs/wakis-env/lib/python3.11/site-packages/pyvista/plotting/picking.py:475: PyVistaDeprecationWarning: `use_mesh` is deprecated. See `use_picker` instead.
  warnings.warn(


Widget(value='<iframe src="http://localhost:38073/index.html?ui=P_0x7fdb8482d490_0&reconnect=auto" class="pyvi…